In [ ]:
!pip install -U datasets
!pip install fsspec==2023.9.2
!pip install datasets
!pip install peft
!pip install -U bitsandbytes


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `1` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when p

In [ ]:
import torch
from transformers import AutoModel
from peft import PeftModel
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoProcessor, BitsAndBytesConfig, AutoModelForImageTextToText,AutoModelForVision2Seq
from peft import LoraConfig
base_model_name = "Qwen/Qwen2-VL-7B-Instruct"

adapters_name = "Dtarget/R49JqwenloraMed"

bnb_config = {
    "load_in_4bit": True,
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_compute_dtype": torch.float16
}

# Load the base model with QLoRA for memory efficiency
modelF = AutoModelForVision2Seq.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    quantization_config=bnb_config  # QLoRA for memory efficiency
)


modelF = PeftModel.from_pretrained(modelF, adapters_name)
modelF = modelF.merge_and_unload()





/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    do_image_splitting=False
)

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

In [ ]:
from datasets import load_dataset


In [ ]:
vqa_rad = load_dataset("flaviagiammarino/vqa-rad")


README.md: 0.00B [00:00, ?B/s]

(…)-00000-of-00001-eb8844602202be60.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

(…)-00000-of-00001-e5bc3d208bb4deeb.parquet:   0%|          | 0.00/10.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1793 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/451 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig
from datasets import load_dataset
from torch.utils.data import Dataset
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score, recall_score
from torch.utils.data import Dataset
import random

class VQADataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        question = self.dataset[idx]['question']
        answer = self.dataset[idx]['answer']
        image = self.dataset[idx]['image']  # Assuming it's a PIL image

        return {
            "image": image,
            "query": {"en": question},
            "answers": [answer]
        }


# Define the MyDataCollator class
class MyDataCollator:
    def __init__(self, processor):
        self.processor = processor
        self.image_token = '<image>'
        self.end_of_utterance_token = '<end_of_utterance>'
        self.pad_token_id = getattr(processor, 'pad_token_id', 0)  # Default to 0 if not found

    def __call__(self, examples):
        texts = []
        for example in examples:
            question = example["query"]['en']
            answer = random.choice(example["answers"])

            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Answer briefly."},
                        {"type": "text", "text": self.image_token},
                        {"type": "text", "text": question}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": answer},
                        {"type": "text", "text": self.end_of_utterance_token}
                    ]
                }
            ]

            text = self.processor.apply_chat_template(messages, add_generation_prompt=False)

            if isinstance(text, list):
                text = " ".join(str(item) for item in text)
            else:
                text = str(text)

            texts.append(text.strip())

        text_batch = self.processor(text=texts, return_tensors="pt", padding=True)

        # Prepare the batch dictionary
        batch = {
            "input_ids": text_batch["input_ids"],
            "attention_mask": text_batch["attention_mask"]
        }

        # Create labels
        labels = batch["input_ids"].clone()
        labels[labels == self.pad_token_id] = -100  # Ignore padding in loss calculation
        batch["labels"] = labels

        return batch



test_dataset = VQADataset(dataset=vqa_rad['test'], processor=processor)

data_collator = MyDataCollator(processor=processor)

In [ ]:
release_memory(modelF)

[None]

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from accelerate.utils import release_memory

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelF.to(device)

predictions = []
ground_truths = []
queries = []
sampled_dataset = random.sample(list(test_dataset), 100)
# Generate text for each example
for example in tqdm(sampled_dataset, desc="Generating answers"):
    image = example["image"]
    query = example["query"]
    ground_truth = example["answers"]

    messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Answer briefly."},
            {"type": "text", "text": processor.image_token},
            {"type": "text", "text": query.get('en', '')}
        ]
    }
]


    text = processor.apply_chat_template(messages, add_generation_prompt=True)

    inputs = processor(
        text=[text.strip()],
        images=[image],
        return_tensors="pt",
        padding=True
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}


    # Generate predictions
    with torch.no_grad():
        generated_ids = modelF.generate(
                **inputs,
                max_new_tokens=3,  # Very short response
                do_sample=False,   # Deterministic outputs
                temperature=0.3,   # No randomness
                top_p=0.0,         # Focus on top token
                repetition_penalty=1.2  # Avoid repetitive outputs
            )

        release_memory(modelF)
        generated_text = processor.batch_decode(
            generated_ids[:, inputs["input_ids"].size(1):], skip_special_tokens=True
        )[0].strip()

    predictions.append(generated_text)
    ground_truths.append(ground_truth)
    queries.append(query)

# Create DataFrame
df = pd.DataFrame({
    "Query": queries,
    "Ground Truth": ground_truths,
    "Prediction": predictions
})






Generating answers: 100%|██████████| 100/100 [03:50<00:00,  2.31s/it]


In [ ]:
!pip install fuzzywuzzy

In [ ]:
import numpy as np
import torch
import pandas as pd
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')


binary_labels = [1 if "yes" in gt.lower() else 0 for gts in ground_truths for gt in gts]  # Convert to binary labels
binary_preds = [1 if "yes" in pred.lower() else 0 for pred in predictions]  # Convert predictions to binary

accuracy_binary = accuracy_score(binary_labels[:len(binary_preds)], binary_preds)

print(f"🌟 Binary Accuracy: {accuracy_binary:.4f}")






🌟 Binary Accuracy: 0.61
